# Databases


## Starting Out: Introduction to SQL and Relational Databases

SQL is a language designed for a very specific purpose: to interact with relational databases. 

- **Database**: A database is a structured collection of data. There are various different ways of structuring the database, and there may or may not be information about the relationship between entities in the database.
- **Query**: A query is a request for data from the database. 
- **Database Management System (DBMS)**: A DBMS is a system of storing and managing databases, including querying the database.
- **Relational Database Management System (RDBMS)**: In an RDBMS, data records are stored in *tables*, each of which has a predefined set of *columns*, the pieces of information captured for each record in a table, and *rows* in the table, where each row has a place to store a value for every column in the table.

Tables, including their columns, column types and relationships with other tables, are defined in a database **schema**. Many times, tables will contain a **primary key**, one or more columns that uniquely define a row. You can think of the primary key as a kind of ID, in which each row is given a unique ID. Tables can also contain  **foreign keys**, which are column(s) that comprise the primary key in another table and, thus, provides a way of matching between multiple tables.

In this notebook, we will use SQL to:
- Select data subsets
- Sum over groups
- Create new tables
- Count distinct values of desired variables
- Order data by chosen variables

### Step 1: Establish a Connection to the Database

The first parameter is the connection to the database. To create a connection, we will use the SQLAlchemy package and tell it which database we want to connect to. Since we are using SQLite for our example notebooks, we will use the 

In [6]:
%defaultDatasource jdbc:sqlite:ncdoc.db

### Formulate Data Query

Depending on what data we are interested in, we can use different queries to pull different data. In this example, we will pull all the content of the offenders data.

__Create a query as a `string` object in Python__

In [7]:
SELECT *
FROM inmate
LIMIT 20;

You should see 20 rows of the `inmate` dataset. Let's go over the basics of this SQL command.

- **SELECT:** We start out with the `SELECT` statement. The `SELECT` statement specifies which variables (columns) you want. 
    - Here, we used `SELECT *`. The "`*`" just says that we want all the variables. 
    - If we wanted a few columns, we would use the column names separated by commas instead of "`*`".


- **FROM:** Now, let's look at the next part of the query, `FROM inmate`. This part of the query specifies the table, `inmate`, from which we want to retrieve the data. Most of your queries will begin in this fashion, describing which columns you want and from which table.


- **LIMIT:** We typically include a `LIMIT` statement at the end of our query so that we don't get overloaded with rows being output. Here, `LIMIT 20` means that we just want the first ten rows. Many times, the `LIMIT` that you want will be higher than 20 -- you might generally prefer to use 1000 or so. Having a `LIMIT` for all queries is highly recommended even if you know only a few rows will be shown, since it acts as a safety precaution against (for example) displaying millions of rows of data.

In this case, we've put everything in one line, but that's not necessary. We could have split the code up into multiple lines, like so:

    SELECT *
    FROM inmate
    LIMIT 20;

This gives the same output as our original query. Generally, once queries start getting longer, breaking up the code into multiple lines can be very helpful in organizing your code and making it easier to read.

Along those lines, note that we used a semi-colon at the end of the query to mark the end of the query. That isn't absolutely necessary here, but it does help mark the end of a query and is required in other applications of SQL, so it's good practice to use it. 

> ### Side note about capitalization
If you notice, we've been using all caps for SQL commands and all lowercase for data table and schema names. This is simply a convention, as SQL is not case sensitive. For example, we could have run `select * from inmate limit 20;` and it would have given us the exact same output as the first query. 

> This does mean you need to be careful when using column names. If your column name has capital letters in it, you need use double quotes (e.g. `"C000"`) to preserve the capitalization. For this reason, you might find that using all lowercase letters in column names is preferable, which is what we've done here.

Now, consider the following query. What do you think it will do?

    SELECT w_geocode, createdate 
    FROM ca_wac_2015
    LIMIT 100;

We've changed the original query by using `w_geocode, createdate` instead of `*`, so we'll only get the values from two columns, `w_geocode` and `createdate`. In addition, we've changed the value after `LIMIT` to be 100 instead of 10, so we'll get the first 100 rows instead of the first 10 rows.

> Note that the `LIMIT` provides one simple way to get a "sample" of data; however, using `LIMIT` does **not provide a _random_** sample. You may get different samples of data than others using just the `LIMIT` clause, but it is just based on what is fastest for the database to return.

### Pull Data from the Database

Now that we have the two parameters (database connection and query), we can pass them to the `pd.read_sql()` function, and obtain the data.

## Conditional Statements

## Joins